# Workshop I - Systems Analysis

## Workshop Definition:

Welcome to the first workshop of Systems Analysis course. Let’s funny me with a biological exercise.

Imagine you have been hired as data analyst in an important biotechnology company.  Your boss, science chief officer, want to get some patterns in genomic data, sometimes called motifs.

Here you will have some tasks in order to complete this workshop:
Create a dummy database of genetic sequences composed of nucleotide bases (A, C, G, T), where each sequence must have between 10 and 20 bases. Your database must be composed for 50.000 genetic sequences.

Get the motifs (must repeated sequence) of size 6 and 8.
Use the Shannon Entropy measurement to filter sequences with not a good variance level.
Get again the motifs of size 6 and 8.

Write some conclusions based on your analysis.
Write any technical concern/decision/difficulty  you think is relevant regarding your work.
You must deliver a full report detailing each one of the previous steps. For steeps 1 to 4 you must describe the algorithms you propose and let an screenshot about the code and the output of the code. I strongly recommend you to use a Jupyter Notebook or a COLAB to write/execute your code.